In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import general_utils as Ugen
import matplotlib.pyplot as plt
import pandas as pd
import math
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob, os

In [4]:
##GET DATA
#GET FILES
main_dir=os.getcwd()
allFiles = glob.glob(main_dir+'/toronto_parking_data'+ "/*.csv")

#LOAD TO DATAFRAME
list_ = []
for file_path in allFiles[-3:]:#allFiles: #START WITH 2015 DATA
    try:
        df = pd.read_csv(file_path,index_col=None,quoting=2,error_bad_lines=False) #quoting=2
        list_.append(df)
        print ('successfully loaded: '+ file_path)
    except Exception as err:
        if 'EOF following escape character' in str(err):
            try:
                df = pd.read_csv(file_path,index_col=None,encoding='utf-16',error_bad_lines=False) #quoting=2
                list_.append(df)
                print ('successfully loaded: '+ file_path)
            except Exception as err:
                print ('NOT loaded: '+ file_path)
                print (str(err))
        else:
            print ('NOT loaded: '+ file_path)
            print (str(err))
tpt_df = pd.concat(list_)
tpt_df.head()

successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_3.csv


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***87325,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE,NaN,NaN,ON
1,***87326,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE,NaN,NaN,ON
2,***09493,20150101,3,PARK ON PRIVATE PROPERTY,30,2,NR,2425 JANE ST,NaN,NaN,ON
3,***67358,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,DUNDAS SQ,E/O,YONGE ST,ON
4,***87327,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,ARMOURY ST,W/O,CENTRE AVE,ON


In [5]:
print (tpt_df.shape)
tpt_df.head()

(2168493, 11)


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***87325,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE,NaN,NaN,ON
1,***87326,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE,NaN,NaN,ON
2,***09493,20150101,3,PARK ON PRIVATE PROPERTY,30,2,NR,2425 JANE ST,NaN,NaN,ON
3,***67358,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,DUNDAS SQ,E/O,YONGE ST,ON
4,***87327,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,ARMOURY ST,W/O,CENTRE AVE,ON


In [6]:
tpt_df=tpt_df.drop(['tag_number_masked','location3','location4','province'],axis=1)
print (tpt_df.shape)
tpt_df.head()

(2168493, 7)


,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2
0,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE
1,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE
2,20150101,3,PARK ON PRIVATE PROPERTY,30,2,NR,2425 JANE ST
3,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,DUNDAS SQ
4,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3,N/S,ARMOURY ST


In [7]:
tpt_fh_df=tpt_df[tpt_df['infraction_code']==15]
tpt_fh_df.location2.value_counts()

##the following are the top grossing fire hydrants....
##multiply each number by $100 to get total revenue from 2012-2015

99 ATLANTIC AVE         438
393 UNIVERSITY AVE      317
33 ELMHURST AVE         283
361 UNIVERSITY AVE      274
5453 YONGE ST           265
112 MERTON ST           237
592 MARKHAM ST          212
500 WELLINGTON ST W     206
17 LILLIAN ST           205
5460 YONGE ST           198
56 THE ESPLANADE        191
5533 YONGE ST           160
150 KILGOUR RD          148
123 SUNNYSIDE AVE       139
17 NIAGARA ST           136
6 SPRING GARDEN AVE     136
21 GLEBE RD E           131
90 KIPPENDAVIE AVE      125
295 RONCESVALLES AVE    124
26 WELLINGTON ST E      120
488 UNIVERSITY AVE      111
43 ELM ST               104
293 SPADINA AVE          99
293 RONCESVALLES AVE     99
DORIS AVE                98
5100 YONGE ST            97
139 WOODWARD AVE         96
20 ARGYLE ST             95
44 BALLIOL ST            94
417 SPADINA RD           91
                       ... 
246 CARLTON ST            1
47 HARVARD AVE            1
5494 YONGES T             1
35 GRANT ST               1
150 GILGORM RD      

In [10]:
##LETS FIND THE HIGHEST GROSSING PERIOD
infraction_ids=list(tpt_df.infraction_code.unique())

# tpt_df.groupby(['infraction_code','location2']).size()
#tpt_df.groupby(['infraction_code','location2']).location2.count()

# tpt_df_mod=tpt_df.iloc[:500000]
tpt_df_mod=tpt_df
tpt_df_mod=tpt_df_mod.sort_values('location2')

# tpt_df_mod['counts']=tpt_df_mod.groupby(['infraction_code','location2']).location2.transform('count')
# tpt_df_mod

In [11]:
tpt_df_mod.shape
# sorted_df.columns.values

(2168493, 7)

In [12]:
#DROP MISSING LOCATIONS
tpt_df_mod=tpt_df_mod[pd.notnull(tpt_df_mod['location2'])]
tpt_df_mod.shape

(2168039, 7)

In [13]:
new_df=pd.DataFrame({'count' : tpt_df_mod.groupby( ['infraction_code','location2'] ).size()}).reset_index()
print(new_df.shape)



(352640, 3)


In [14]:
##now calculate total amount...
##create dict of infraction_code:fine_amount
fine_amount_dict={ic:{'fine_amt':tpt_df[tpt_df['infraction_code']==ic].set_fine_amount.median(),
                      'fine_descp':tpt_df[tpt_df['infraction_code']==ic].infraction_description.iloc[0]}    
                          for ic in infraction_ids}


def calc_fine_sum(df):
    return df['count']*fine_amount_dict[df['infraction_code']]['fine_amt']

def get_fine_descrp(df):
    return fine_amount_dict[df['infraction_code']]['fine_descp']

new_df['fine_sum']=new_df.apply(calc_fine_sum,axis=1)
new_df['fine_descrp']=new_df.apply(get_fine_descrp,axis=1)

In [15]:
new_df.sort_values('fine_sum',ascending=False)

,infraction_code,location2,count,fine_sum,fine_descrp
299275,347,1090 DON MILLS RD,1889,472250,PARK IN A FIRE ROUTE
304354,367,410 COLLEGE ST,604,271800,STND ONSTRT ACCESSIBLE NO PRMT
287935,257,1 BRIMLEY RD S,2267,226700,PARK CONTRARY POSTED CONDITION
301048,347,45 OVERLEA BLVD,787,196750,PARK IN A FIRE ROUTE
43379,3,2075 BAYVIEW AVE,5231,156930,PARK ON PRIVATE PROPERTY
302012,355,1090 DON MILLS RD,313,140850,PARK IN ACCESSIBLE NO PERMIT
303716,363,35 BALMUTO ST,309,139050,PARK ONSTRT ACCESSIBLE NO PRMT
72668,5,20 EDWARD ST,3344,133760,PARK-SIGNED HWY-PROHIBIT DY/TM
304266,367,18 GRENVILLE ST,293,131850,STND ONSTRT ACCESSIBLE NO PRMT
47259,3,3401 DUFFERIN ST,4341,130230,PARK ON PRIVATE PROPERTY


In [16]:
new_df[new_df['location2']=='939 LAWRENCE AVE E']

,infraction_code,location2,count,fine_sum,fine_descrp
301819,347,939 LAWRENCE AVE E,1,250,PARK IN A FIRE ROUTE


In [14]:
tpt_df.columns

Index(['date_of_infraction', 'infraction_code', 'infraction_description',
       'set_fine_amount', 'time_of_infraction', 'location1', 'location2'],
      dtype='object')

In [22]:
##get unique values of infraction times
print(tpt_df.time_of_infraction.describe())

tpt_df[tpt_df['time_of_infraction']>0].sort('time_of_infraction',ascending=True).head(100)

count    2167063.000000
mean        1188.662348
std          604.542799
min            0.000000
25%          833.000000
50%         1232.000000
75%         1633.000000
max         2359.000000
Name: time_of_infraction, dtype: float64


/Users/whitesi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2
0,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1,OPP,11 CENTRE AVE
513415,20150401,29,PARK PROHIBITED TIME NO PERMIT,30,1,NR,121 SCOLLARD ST
513414,20150401,406,PARK-VEH. W/O VALID ONT PLATE,40,1,NR,5330 YONGE ST
513413,20150401,29,PARK PROHIBITED TIME NO PERMIT,30,1,NR,26 RAGLAN AVE
513412,20150401,3,PARK ON PRIVATE PROPERTY,30,1,AT,333 SIDNEY BELSEY CRES
506798,20150331,29,PARK PROHIBITED TIME NO PERMIT,30,1,NR,25 ROCKVALE AVE
500162,20150330,3,PARK ON PRIVATE PROPERTY,30,1,AT,40 PANORAMA CRT
500161,20150330,29,PARK PROHIBITED TIME NO PERMIT,30,1,NR,111 RUNNYMEDE RD
500160,20150330,9,STOP-SIGNED HWY-PROHIBIT TM/DY,60,1,OPP,15 ASQUITH AVE
408924,20151118,29,PARK PROHIBITED TIME NO PERMIT,30,1,NR,THIRTEENTH ST


In [50]:
import requests

req_url='https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'
api_key = Ugen.ConfigSectionMap('googlemaps')['key']
# api_response = requests.get(req_url.format(address,api_key))
# api_response_dict = api_response.json()


def get_coords(df):
    address=df['location2']+' Toronto, Ontario, Canada'
    try:
        response = requests.get(req_url.format(address,api_key))
        response_dict = response.json()
        coords=response_dict['results'][0]['geometry']['location']
        if response_dict['status'] == 'OK':
#             print (response_dict)
            return coords['lat'],coords['lng'],response_dict['results'][0]['formatted_address'].replace(',','')
    except Exception as err:
        print (str(err))
        pass
    return None,None,None

##OPTION 2
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()

# def get_coords(df):
#     street=df['location2']
#     try:
#         location = geolocator.geocode(street+" Toronto Ontario Canada")
#         if location.latitude and location.longitude:
#             return {'lon':location.longitude,'lat':location.latitude}
#     except Exception as err:
#         print (str(err))
#         pass
#     return



new2_df=new_df[new_df['infraction_code']==1]
new2_df.shape
new2_df['lat'],new2_df['lng'],new2_df['address']=zip(*new2_df.apply(get_coords,axis=1))

/Users/whitesi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
new2_df.head()

,infraction_code,location2,count,fine_sum,fine_descrp,lat,lng,address
0,1,1 COLLINSON BLVD,4,120,PARK FAIL TO DEPOSIT FEE METER,43.737834,-79.437400,1 Collinson Blvd North York ON M3H 3B7 Canada
1,1,1 FAIRHOLME AVE,2,60,PARK FAIL TO DEPOSIT FEE METER,43.718255,-79.430296,1 Fairholme Ave North York ON M6B 2W4 Canada
2,1,1 HOWLAND AVE,1,30,PARK FAIL TO DEPOSIT FEE METER,43.665993,-79.408603,1 Howland Ave Toronto ON M5R 3B2 Canada
3,1,1 KEEWATIN AVE,2,60,PARK FAIL TO DEPOSIT FEE METER,43.712331,-79.399280,1 Keewatin Ave Toronto ON M4P 1Z9 Canada
4,1,1 WESTMORELAND AVE,1,30,PARK FAIL TO DEPOSIT FEE METER,43.661033,-79.430324,1 Westmoreland Ave Toronto ON M6H 2Z6 Canada


In [52]:
new2_df.to_csv('./tpt_sample.csv')